In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
import gc
import pickle

%matplotlib inline

pd.set_option('display.max_rows', 304)

import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Some notes/ideas from EDA:
* Looks like missing values were replaced with 0 (including in target!).
* Feature for number of time_ids since this investment_id started?
* Submit one version for scoring with lag and time features, and one without
* Note: if doing lag features, will need to be careful to combine test data with train data in order to make sure lag features are included.
* All features are scaled already (including target) - roughly mean of 0 and standard deviation 1. However, many features are bimodal, or skewed.
* All features have low correlation with target, but there are some that correlate with each other.
* Initial validation: time-based
* Validation goal: want validation score to be similar to leaderboard score

In [ ]:
df = pd.read_parquet("../input/ubiquant-parquet/train_low_mem.parquet")
print (len(df.index))

In [ ]:
# create validation dataset before filtering so that validation data is always the same
'''
features = ['f_'+str(i) for i in range(300)]
val_time_cutoff = np.floor(df['time_id'].quantile(0.75))
X_val = df.loc[df['time_id']>val_time_cutoff, ['investment_id']+features].values
Y_val = df.loc[df['time_id']>val_time_cutoff, 'target'].values

# save validation data to file
np.save('X_val.npy', X_val)
np.save('Y_val.npy', Y_val)

del X_val
del Y_val
gc.collect()
'''

features = ['f_'+str(i) for i in range(300)]
investment_id = df['investment_id'].copy()
X = df[['investment_id','time_id']+features].values.astype('float16')
Y = df['target'].values
np.save('X.npy', X)
np.save('Y.npy', Y)

del X, Y
gc.collect()

In [ ]:

features = ['f_'+str(i) for i in range(300)]

# segement data for time-based cross-fold validation
time_cutoff_50 = np.floor(df['time_id'].quantile(0.5))
time_cutoff_60 = np.floor(df['time_id'].quantile(0.6))
time_cutoff_70 = np.floor(df['time_id'].quantile(0.7))
time_cutoff_80 = np.floor(df['time_id'].quantile(0.8))
time_cutoff_90 = np.floor(df['time_id'].quantile(0.9))

X_0_50 = df.loc[df['time_id']<=time_cutoff_50, ['investment_id','time_id']+features].values.astype('float16')
Y_0_50 = df.loc[df['time_id']<=time_cutoff_50, 'target'].values
np.save('X_0_50.npy', X_0_50)
np.save('Y_0_50.npy', Y_0_50)
del X_0_50, Y_0_50

X_50_60 = df.loc[(df['time_id']>time_cutoff_50) & (df['time_id']<=time_cutoff_60), ['investment_id','time_id']+features].values.astype('float16')
Y_50_60 = df.loc[(df['time_id']>time_cutoff_50) & (df['time_id']<=time_cutoff_60), 'target'].values
np.save('X_50_60.npy', X_50_60)
np.save('Y_50_60.npy', Y_50_60)
del X_50_60, Y_50_60

X_60_70 = df.loc[(df['time_id']>time_cutoff_60) & (df['time_id']<=time_cutoff_70), ['investment_id','time_id']+features].values.astype('float16')
Y_60_70 = df.loc[(df['time_id']>time_cutoff_60) & (df['time_id']<=time_cutoff_70), 'target'].values
np.save('X_60_70.npy', X_60_70)
np.save('Y_60_70.npy', Y_60_70)
del X_60_70, Y_60_70

X_70_80 = df.loc[(df['time_id']>time_cutoff_70) & (df['time_id']<=time_cutoff_80), ['investment_id','time_id']+features].values.astype('float16')
Y_70_80 = df.loc[(df['time_id']>time_cutoff_70) & (df['time_id']<=time_cutoff_80), 'target'].values
np.save('X_70_80.npy', X_70_80)
np.save('Y_70_80.npy', Y_70_80)
del X_70_80, Y_70_80

X_80_90 = df.loc[(df['time_id']>time_cutoff_80) & (df['time_id']<=time_cutoff_90), ['investment_id','time_id']+features].values.astype('float16')
Y_80_90 = df.loc[(df['time_id']>time_cutoff_80) & (df['time_id']<=time_cutoff_90), 'target'].values
np.save('X_80_90.npy', X_80_90)
np.save('Y_80_90.npy', Y_80_90)
del X_80_90, Y_80_90

X_90_100 = df.loc[df['time_id']>time_cutoff_90, ['investment_id','time_id']+features].values.astype('float16')
Y_90_100 = df.loc[df['time_id']>time_cutoff_90, 'target'].values
np.save('X_90_100.npy', X_90_100)
np.save('Y_90_100.npy', Y_90_100)
del X_90_100, Y_90_100

gc.collect()

In [ ]:
## for now, just use a sub-sample to do some exploration
#df = df.sample(frac=0.05, random_state=26)

# sample using investment_id
#sampled_ids = pd.Series(df['investment_id'].unique()).sample(frac=0.35, random_state=26)
#df = df.loc[(df['investment_id'].isin(sampled_ids)) & (df['time_id']<=val_time_cutoff)]

In [ ]:
'''
df = df.loc[df['time_id']<=val_time_cutoff]
X_train = df[['investment_id']+features].values
Y_train = df['target'].values

# save train data to file
np.save('X_train.npy', X_train)
np.save('Y_train.npy', Y_train)

del X_train
del Y_train
gc.collect()
'''

# Feature Engineering and Validation Set

In [ ]:
'''def _add_lag(df, cols, lag, groupCol):
    new_cols = [col+'_last'+str(lag) for col in cols]
    df[new_cols] = df.groupby(groupCol)[cols].fillna(method='ffill').transform(lambda x: x.rolling(window=lag).mean())
    df[new_cols] = df.groupby(groupCol)[new_cols].shift(1)

    return df, new_cols'''

In [ ]:
#df = df.sort_values(by=['time_id','investment_id'])

In [ ]:
#features = ['f_'+str(i) for i in range(300)]

In [ ]:
'''#df = df.drop(new_cols, 1) #if want to get a new lag and remove old lag columns
df = df.copy() #fixes gaps in index warning
df, new_cols = _add_lag(df, features + ['target'], 1, 'investment_id')
features += new_cols
#features = new_cols
df = df.fillna(0.)'''

Initial strategy will be to hold out the highest ~25% of data in terms of time_id as a validation set

In [ ]:
'''
val_time_cutoff = np.floor(df['time_id'].quantile(0.75))
print (val_time_cutoff)
X_train = df.loc[df['time_id']<=val_time_cutoff, features].values
Y_train = df.loc[df['time_id']<=val_time_cutoff, 'target'].values
X_val = df.loc[df['time_id']>val_time_cutoff, features].values
Y_val = df.loc[df['time_id']>val_time_cutoff, 'target'].values'''

In [ ]:
del df
gc.collect()

# Training

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold, StratifiedKFold
from scipy.stats import pearsonr
import lightgbm as lgb
import random

from keras.layers.core import Dense, Activation, Dropout
from keras.models import Sequential, load_model, Model
from keras.layers import Input, IntegerLookup, Embedding, Concatenate, Reshape
from keras import backend as K
import keras
import tensorflow as tf

In [ ]:
pca = PCA(n_components=70, random_state=26)

X_train = np.load('X_train.npy')
X_train_pca = pca.fit_transform(X_train)
del X_train

np.save('X_train_pca.npy', X_train_pca)
del X_train_pca

X_val = np.load('X_val.npy')
X_val_pca = pca.transform(X_val)
del X_val

np.save('X_val_pca.npy', X_val_pca)
del X_val_pca

with open('pca.pkl', 'wb') as f:
    pickle.dump(pca, f)
    
del pca
gc.collect()

In [ ]:
lr = LinearRegression()

#lr.fit(X_train, Y_train)
#print ('Validation Pearson r: {}'.format(str(pearsonr(lr.predict(X_val), Y_val)[0])))
X_train_pca = np.load('X_train_pca.npy')
Y_train = np.load('Y_train.npy')
lr.fit(X_train_pca, Y_train)

with open('lr.pkl', 'wb') as f:
    pickle.dump(lr, f)
    
del X_train_pca
del Y_train

X_val_pca = np.load('X_val_pca.npy')
Y_val = np.load('Y_val.npy')
print ('Validation Pearson r: {}'.format(str(pearsonr(lr.predict(X_val_pca), Y_val)[0])))

del X_val_pca
del Y_val
del lr

gc.collect()

In [ ]:
params = {
    'boosting_type': 'gbdt',
    'metric': 'mse',
    'objective': 'regression',
    'n_jobs': -1,
    'seed': 26
}

kfold = StratifiedKFold(3, shuffle=True, random_state=26)
fold_scores = []
model_i = 1
for train_index, val_index in kfold.split(np.zeros(len(investment_id.index)), investment_id):
    X = np.load('X.npy')[:,1:]
    Xt = X[train_index]
    Xv = X[val_index]
    del X
    Y = np.load('Y.npy')
    Yt = Y[train_index]
    Yv = Y[val_index]
    del Y
    gc.collect()
    
    gb = lgb.train(params, lgb.Dataset(Xt, Yt))
    fold_scores.append(pearsonr(gb.predict(Xv), Yv)[0])
    print (fold_scores)
    
    with open('gb_{}.pkl'.format(str(model_i)), 'wb') as f:
        pickle.dump(gb, f)
    del gb, Xv, Xt, Yv, Yt
    gc.collect()
    model_i += 1

print ('Mean Pearson r: '+str(np.mean(fold_scores)))

In [ ]:
def correlation_coefficient_loss(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(tf.multiply(xm,ym))
    r_den = K.sqrt(tf.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
    r = r_num / r_den

    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return 1 - K.square(r)

def create_model_nn():
    features_input = Input((len(features),))
    
    x = Dropout(0.2)(features_input)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu')(x)
    out = Dense(1)(x)
    
    model = keras.Model(inputs=[features_input], outputs=[out])
    model.compile(loss=correlation_coefficient_loss, optimizer=tf.optimizers.Adam(0.001))
    return model

def create_model_nn2():
    features_input = Input((len(features),))
    
    x = Dense(2048, activation='gelu')(x)
    x = Dense(1024, activation='relu')(x)
    x = Dense(512, activation='gelu')(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    out = Dense(1)(x)
    
    model = keras.Model(inputs=[features_input], outputs=[out])
    model.compile(loss=correlation_coefficient_loss, optimizer=tf.optimizers.Adam(0.001))
    return model

features = ['f_'+str(i) for i in range(300)]

kfold = StratifiedKFold(5, shuffle=True, random_state=26)
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)

fold_scores = []

model_i = 1
for train_index, val_index in kfold.split(np.zeros(len(investment_id.index)), investment_id):
    X = np.load('X.npy')
    Xt = X[train_index]
    Xv = X[val_index]
    del X
    Y = np.load('Y.npy')
    Yt = Y[train_index]
    Yv = Y[val_index]
    del Y
    gc.collect()
    
    nn = create_model_nn()
    nn.fit(Xt[:,1:], Yt, epochs=50, batch_size=1024, validation_data=(Xv[:,1:], Yv), callbacks=[early_stop])
    fold_scores.append(pearsonr(nn.predict(Xv[:,1:])[:,0], Yv)[0])
    nn.save('nn_{}.h5'.format(str(model_i)))
    del nn
    gc.collect()
    
    print (fold_scores)
    
    del Xv, Xt, Yv, Yt
    gc.collect()
    model_i += 1

print ('Mean Pearson r: '+str(np.mean(np.abs(fold_scores))))

5-fold stratified CV:

4096, 512, 32 (with all 0.2s) relu: 0.1864 val, LB 0.144

2048 relu, 1024 gelu, 512 gelu, 128 gelu, 32 relu (no dropouts): 0.1799 val, LB 0.145

In [ ]:
params = {
    'boosting_type': 'gbdt',
    'metric': 'mse',
    'objective': 'regression',
    'n_jobs': -1,
    'seed': 26
}

def correlation_coefficient_loss(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(tf.multiply(xm,ym))
    r_den = K.sqrt(tf.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
    r = r_num / r_den

    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return 1 - K.square(r)

def create_model_nn(ids):
    id_input = Input((1,))
    features_input = Input((len(features),))
    
    ids = np.unique(ids)
    x_ids = IntegerLookup(max_tokens=len(ids)+1, vocabulary=ids)(id_input)
    x_ids = Embedding(len(ids)+1, 64, input_length=1)(x_ids)
    x_ids = Reshape((-1,))(x_ids)
    x_ids = Dense(64, activation='swish')(x_ids)
    x_ids = Dense(64, activation='swish')(x_ids)
    x_ids = Dense(64, activation='swish')(x_ids)
    
    x_features = Dense(512, activation='swish')(features_input)
    x_features = Dense(512, activation='swish')(x_features)
    x_features = Dense(512, activation='swish')(x_features)
    x_features = Dense(512, activation='swish')(x_features)
    
    concat = Concatenate(axis=1)([x_features, x_ids])
    
    x = Dropout(0.2)(concat)
    x = Dense(512, activation='swish')(x)
    x = Dropout(0.2)(x)
    x = Dense(128, activation='swish')(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='swish')(x)
    out = Dense(1)(x)
    
    model = keras.Model(inputs=[id_input, features_input], outputs=[out])
    model.compile(loss=correlation_coefficient_loss, optimizer=tf.optimizers.Adam(0.001))
    return model

def create_model_nn2():
    features_input = Input((len(features),))
    
    x = Dropout(0.2)(features_input)
    x = Dense(2048, activation='swish')(x)
    x = Dropout(0.2)(x)
    x = Dense(1024, activation='swish')(x)
    x = Dropout(0.2)(x)
    x = Dense(512, activation='swish')(x)
    x = Dropout(0.2)(x)
    x = Dense(128, activation='swish')(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='swish')(x)
    out = Dense(1)(x)
    
    model = keras.Model(inputs=[features_input], outputs=[out])
    model.compile(loss=correlation_coefficient_loss, optimizer=tf.optimizers.Adam(0.001))
    return model

features = ['f_'+str(i) for i in range(300)]

kfold = StratifiedKFold(5, shuffle=True, random_state=26)
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)

fold_scores = []

model_i = 1
for train_index, val_index in kfold.split(np.zeros(len(investment_id.index)), investment_id):
    X = np.load('X.npy')
    Xt = X[train_index]
    Xv = X[val_index]
    del X
    Y = np.load('Y.npy')
    Yt = Y[train_index]
    Yv = Y[val_index]
    del Y
    gc.collect()
    
    nn = create_model_nn(Xt[:,0])
    nn.fit([Xt[:,0], Xt[:,1:]], Yt, epochs=50, batch_size=1024, validation_data=([Xv[:,0], Xv[:,1:]], Yv), callbacks=[early_stop])
    #fold_scores.append(pearsonr(nn.predict([Xv[:,0], Xv[:,1:]])[:,0], Yv)[0])
    preds_nn = nn.predict([Xv[:,0], Xv[:,1:]])[:,0]
    nn.save('nn_{}.h5'.format(str(model_i)))
    del nn
    pr_1 = pearsonr(preds_nn, Yv)[0]
    if pr_1<0:
        preds_nn *= -1
    gc.collect()
    
    nn2 = create_model_nn2()
    nn2.fit(Xt[:,1:], Yt, epochs=50, batch_size=1024, validation_data=(Xv[:,1:], Yv), callbacks=[early_stop])
    #fold_scores.append(pearsonr(nn.predict(Xv[:,1:])[:,0], Yv)[0])
    
    #gb = lgb.train(params, lgb.Dataset(Xt[:,1:], Yt))
    preds_nn2 = nn2.predict(Xv[:,1:])[:,0]
    nn2.save('nn2_{}.h5'.format(str(model_i)))
    del nn2
    pr_2 = pearsonr(preds_nn2, Yv)[0]
    if pr_2<0:
        preds_nn2 *= -1
    gc.collect()
    preds_combined = (preds_nn + preds_nn2)/2.
    fold_scores.append([pr_1, pr_2, pearsonr(preds_combined, Yv)[0]])
    
    print (fold_scores)
    
    del Xv, Xt, Yv, Yt
    gc.collect()
    model_i += 1

print ('Mean Pearson r: '+str(np.mean(np.abs(fold_scores))))

Scores
Val scheme: 3 stratified folds on investment_id, mean pearson r across the folds

starting architecture: 0.1772, LB 0.137

with 1024 x_feature layers and dropout over regularizers: 0.1812/0.1743, LB 0.143

KEEP dropout over regularizers: 0.1728

KEEP with 4X 512 x_feature layers: 0.1815/0.1800/0.1813, LB 0.137

nn2, 2048, 1024, 512, 128, 32 with 0.2 dropouts: 0.1826, LB 0.140

lgbm (with investment_id): 0.1709, LB 0.132

# Ensembling

In [ ]:
#del X_train_pca, X_train
#gc.collect()

In [ ]:
with open('lr.pkl', 'rb') as f:
    lr = pickle.load(f)
    
with open('gb.pkl', 'rb') as f:
    gb = pickle.load(f)
    
nn = load_model('nn.h5')

In [ ]:
# try just mean of previous predictions
X_val_pca = np.load('X_val_pca.npy')
preds_lr = lr.predict(X_val_pca)
del X_val_pca

X_val = np.load('X_val.npy')
preds_gb = gb.predict(X_val)
preds_nn = nn.predict(X_val)[:,0]
del X_val

preds = (preds_lr + preds_gb + preds_nn)/3

Y_val = np.load('Y_val.npy')
print ('Validation Pearson r: {}'.format(str(pearsonr(preds, Y_val)[0])))

# Final Train on Train+Val

In [ ]:
pca = PCA(n_components=70, random_state=26)

X_train = np.load('X_train.npy')
X_val = np.load('X_val.npy')
X = np.concatenate([X_train, X_val])
del X_train, X_val
np.save('X.npy', X)
X_pca = pca.fit_transform(X)
del X

np.save('X_pca.npy', X_pca)

with open('pca.pkl', 'wb') as f:
    pickle.dump(pca, f)
    
del pca
gc.collect()

In [ ]:
lr = LinearRegression()

Y_train = np.load('Y_train.npy')
Y_val = np.load('Y_val.npy')
Y = np.concatenate([Y_train, Y_val])
del Y_train, Y_val
np.save('Y.npy', Y)
lr = lr.fit(X_pca, Y)
del X_pca

with open('lr.pkl', 'wb') as f:
    pickle.dump(lr, f)
    
del lr
gc.collect()

In [ ]:
X = np.load('X.npy')
Y = np.load('Y.npy')
gb = lgb.train(params, lgb.Dataset(X[:,1:], Y))
del X, Y

with open('gb.pkl', 'wb') as f:
    pickle.dump(gb, f)

del gb
gc.collect()

In [ ]:
def correlation_coefficient_loss(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(tf.multiply(xm,ym))
    r_den = K.sqrt(tf.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
    r = r_num / r_den

    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return 1 - K.square(r)

def create_model_nn(ids):
    id_input = Input((1,))
    features_input = Input((len(features),))
    
    ids = np.unique(ids)
    x_ids = IntegerLookup(max_tokens=len(ids)+1, vocabulary=ids)(id_input)
    x_ids = Embedding(len(ids)+1, 64, input_length=1)(x_ids)
    x_ids = Reshape((-1,))(x_ids)
    x_ids = Dense(64, activation='swish')(x_ids)
    x_ids = Dense(64, activation='swish')(x_ids)
    x_ids = Dense(64, activation='swish')(x_ids)
    
    x_features = Dense(512, activation='swish')(features_input)
    x_features = Dense(512, activation='swish')(x_features)
    x_features = Dense(512, activation='swish')(x_features)
    x_features = Dense(512, activation='swish')(x_features)
    
    concat = Concatenate(axis=1)([x_features, x_ids])
    
    x = Dropout(0.2)(concat)
    x = Dense(512, activation='swish')(x)
    x = Dropout(0.2)(x)
    x = Dense(128, activation='swish')(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='swish')(x)
    out = Dense(1)(x)
    
    model = keras.Model(inputs=[id_input, features_input], outputs=[out])
    model.compile(loss=correlation_coefficient_loss, optimizer=tf.optimizers.Adam(0.001))
    return model

features = ['f_'+str(i) for i in range(300)]
Y = np.load('Y.npy')

kfold = StratifiedKFold(5, shuffle=True, random_state=26)
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, restore_best_weights=True)

fold_scores = []

model_i = 1
for train_index, val_index in kfold.split(np.zeros(len(investment_id.index)), investment_id):
    X = np.load('X.npy')
    Xt = X[train_index]
    Xv = X[val_index]
    del X
    Y = np.load('Y.npy')
    Yt = Y[train_index]
    Yv = Y[val_index]
    del Y
    gc.collect()
    
    nn = create_model_nn(Xt[:,0])
    nn.fit([Xt[:,0], Xt[:,1:]], Yt, epochs=50, batch_size=1024, validation_data=([Xv[:,0], Xv[:,1:]], Yv), callbacks=[early_stop])
    fold_scores.append(pearsonr(nn.predict([Xv[:,0], Xv[:,1:]])[:,0], Yv)[0])
    print (fold_scores)
    
    nn.save('nn_{}.h5'.format(str(model_i)))
    del nn
    gc.collect()
    model_i += 1

print ('Mean Pearson r: '+str(np.mean(np.abs(fold_scores))))
del Xv, Xt, Yv, Yt

In [ ]:
from typing import Tuple

class GroupTimeSeriesSplit:
    """
    Custom class to create a Group Time Series Split. We ensure
    that the time id values that are in the testing data are not a part
    of the training data & the splits are temporal
    """
    def __init__(self, n_folds: int, holdout_size: int, groups: str) -> None:
        self.n_folds = n_folds
        self.holdout_size = holdout_size
        self.groups = groups

    def split(self, X) -> Tuple[np.array, np.array]:
        # Take the group column and get the unique values
        unique_time_ids = np.unique(self.groups.values)

        # Split the time ids into the length of the holdout size
        # and reverse so we work backwards in time. Also, makes
        # it easier to get the correct time_id values per
        # split
        array_split_time_ids = np.array_split(
            unique_time_ids, len(unique_time_ids) // self.holdout_size
        )[::-1]

        # Get the first n_folds values
        array_split_time_ids = array_split_time_ids[:self.n_folds]

        for time_ids in array_split_time_ids:
            # Get test index - time id values that are in the time_ids
            test_condition = X['time_id'].isin(time_ids)
            test_index = X.loc[test_condition].index

            # Get train index - The train index will be the time
            # id values right up until the minimum value in the test
            # data - we can also add a gap to this step by
            # time id < (min - gap)
            train_condition = X['time_id'] < (np.min(time_ids))
            train_index = X.loc[train_condition].index

            yield train_index, test_index

In [ ]:
features = ['f_'+str(i) for i in range(300)]

def correlation_coefficient_loss(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(tf.multiply(xm,ym))
    r_den = K.sqrt(tf.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
    r = r_num / r_den

    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return 1 - K.square(r)

def create_model_nn():
    features_input = Input((len(features),))
    
    #x = Dropout(0.2)(features_input)
    x = Dense(2048, activation='relu')(features_input)
    x = Dense(1024, activation='gelu')(x)
    x = Dense(512, activation='gelu')(x)
    #x = Dropout(0.2)(x)
    x = Dense(128, activation='gelu')(x)
    #x = Dropout(0.2)(x)
    x = Dense(32, activation='relu')(x)
    out = Dense(1)(x)
    
    model = keras.Model(inputs=[features_input], outputs=[out])
    model.compile(loss=correlation_coefficient_loss, optimizer=tf.optimizers.Adam(0.001))
    return model

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
#X = np.load('X.npy')
#Y = np.load('Y.npy')

df = pd.read_parquet("../input/ubiquant-parquet/train_low_mem.parquet")
gtss = GroupTimeSeriesSplit(n_folds=3, holdout_size=200, groups=df['time_id'])
#del X
fold_scores = []
i = 1
for fold, (train_index, val_index) in enumerate(gtss.split(df)):
    if i==1:
        del df
    X = np.load('X.npy')
    Xt = X[train_index]
    Xv = X[val_index]
    del X
    Y = np.load('Y.npy')
    Yt = Y[train_index]
    Yv = Y[val_index]
    del Y
    gc.collect()
    
    nn = create_model_nn()
    nn.fit(Xt[:,2:], Yt, epochs=50, batch_size=1024, validation_data=(Xv[:,2:], Yv), callbacks=[early_stop])
    preds_nn = nn.predict(Xv[:,2:])[:,0]
    nn.save('nn_{}.h5'.format(str(i)))
    del nn
    gc.collect()
    
    df_val = pd.DataFrame()
    df_val['preds'] = preds_nn
    df_val['time_id'] = Xv[:,1]
    df_val['Y'] = Yv
    
    r = df_val.groupby('time_id').apply(lambda x: pearsonr(df_val['Y'], df_val['preds'])[0]).mean()
    print (r)
    fold_scores.append(r)
    print (fold_scores)
    gc.collect()
    i += 1

print ('Mean Pearson r: '+str(np.mean(np.abs(fold_scores))))
del Xv, Xt, Yv, Yt

nn = create_model_nn()
X = np.load('X.npy')
Y = np.load('Y.npy')
nn.fit(X[:,2:], Y, epochs=4, batch_size=1024)
nn.save('nn.h5')
preds_nn = nn.predict(X[1000000:,2:])[:,0]
del nn
df_val = pd.DataFrame()
df_val['preds'] = preds_nn
df_val['time_id'] = X[1000000:,1]
df_val['Y'] = Y[1000000:]
r = df_val.groupby('time_id').apply(lambda x: pearsonr(df_val['Y'], df_val['preds'])[0]).mean()
print (r)

del X, Y, preds_nn, df_val
gc.collect()

3-fold group time fold CV (val score is mean of fold_scores only, not full train)

4096, 512, 32 (with all 0.2s) relu: (-0.1342, 0.1272, -0.1194, -0.1790), mean 0.1269, LB 0.1389

2048 relu, 1024 gelu, 512 gelu, 128 gelu, 32 relu (no dropouts):

original architecture with id_lookup layer:

5X 512, 32 (with all 0.2s) relu: (0.1399, -0.1297, 0.1245, 0.1764), mean 0.1314, LB 0.1375

10X 256, 32 (with all 0.2s) relu: (-0.1321, -0.1253, -0.1170, -0.1536), mean 0.1248

10X 256, 32 (with all 0.4s) relu: (0.1389, -0.1214, -0.1187, -0.1463), mean 0.1263, LB 0.1353

In [ ]:
def correlation_coefficient_loss(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(tf.multiply(xm,ym))
    r_den = K.sqrt(tf.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
    r = r_num / r_den

    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return 1 - K.square(r)

def create_model_nn():
    features_input = Input((len(features),))
    
    x = Dropout(0.4)(features_input)
    #x = Dense(2048, activation='relu')(features_input)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(32, activation='relu')(x)
    out = Dense(1)(x)
    
    model = keras.Model(inputs=[features_input], outputs=[out])
    model.compile(loss=correlation_coefficient_loss, optimizer=tf.optimizers.Adam(0.001))
    return model

def create_model_nn2(ids):
    id_input = Input((1,))
    features_input = Input((len(features),))
    
    ids = np.unique(ids)
    x_ids = IntegerLookup(max_tokens=len(ids)+1, vocabulary=ids)(id_input)
    x_ids = Embedding(len(ids)+1, 64, input_length=1)(x_ids)
    x_ids = Reshape((-1,))(x_ids)
    x_ids = Dense(64, activation='swish')(x_ids)
    x_ids = Dense(64, activation='swish')(x_ids)
    x_ids = Dense(64, activation='swish')(x_ids)
    
    x_features = Dense(256, activation='swish')(features_input)
    x_features = Dense(256, activation='swish')(x_features)
    x_features = Dense(256, activation='swish')(x_features)
    
    concat = Concatenate(axis=1)([x_features, x_ids])
    
    x = Dense(512, activation='swish', kernel_regularizer='l2')(concat)
    x = Dense(128, activation='swish', kernel_regularizer='l2')(x)
    x = Dense(32, activation='swish', kernel_regularizer='l2')(x)
    out = Dense(1)(x)
    
    model = keras.Model(inputs=[id_input, features_input], outputs=[out])
    model.compile(loss=correlation_coefficient_loss, optimizer=tf.optimizers.Adam(0.001))
    return model

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

#filenames = ['50_60','60_70','70_80','80_90','90_100']
filenames = ['70_80','80_90','90_100']

fold_scores = []
for i in range(3):
    if i==0:
        #X = np.load('X_0_50.npy')[:,1:]
        X = np.load('X_0_50.npy')
        Y = np.load('Y_0_50.npy')
        #X = np.concatenate([X, np.load('X_50_60.npy')[:,1:]])
        X = np.concatenate([X, np.load('X_50_60.npy')])
        Y = np.concatenate([Y, np.load('Y_50_60.npy')])
        #X = np.concatenate([X, np.load('X_60_70.npy')[:,1:]])
        X = np.concatenate([X, np.load('X_60_70.npy')])
        Y = np.concatenate([Y, np.load('Y_60_70.npy')])
        
    #Xv = np.load('X_{}.npy'.format(filenames[i]))[:,1:]
    Xv = np.load('X_{}.npy'.format(filenames[i]))
    Yv = np.load('Y_{}.npy'.format(filenames[i]))
    
    #nn = create_model_nn()
    #nn.fit(X[:,2:], Y, epochs=50, batch_size=1024, validation_data=(Xv[:,2:], Yv), callbacks=[early_stop])
    #preds_nn = nn.predict(Xv[:,2:])[:,0]
    #nn.save('nn_{}.h5'.format(str(i+1)))
    #del nn
    
    nn = create_model_nn2(X[:,0])
    nn.fit([X[:,0], X[:,2:]], Y, epochs=50, batch_size=1024, validation_data=([Xv[:,0], Xv[:,2:]], Yv), callbacks=[early_stop])
    preds_nn = nn.predict([Xv[:,0], Xv[:,2:]])[:,0]
    nn.save('nn_{}.h5'.format(str(i+1)))
    del nn
    
    gc.collect()
    
    df_val = pd.DataFrame()
    df_val['preds'] = preds_nn
    df_val['time_id'] = Xv[:,1]
    df_val['Y'] = Yv
    
    #fold_scores.append(pearsonr(nn.predict(Xv)[:,0], Yv)[0])
    r = df_val.groupby('time_id').apply(lambda x: pearsonr(df_val['Y'], df_val['preds'])[0]).mean()
    print (r)
    #print (pearsonr(preds_nn2, Yv)[0])
    #if pearsonr(preds_nn, Yv)[0]<0:
    #    preds_nn *= -1
    #if pearsonr(preds_nn2, Yv)[0]<0:
    #    preds_nn2 *= -1
    #fold_scores.append(pearsonr((preds_nn + preds_nn2)/2., Yv)[0])
    fold_scores.append(r)
    print (fold_scores)
    
    X = np.concatenate([X, Xv])
    Y = np.concatenate([Y, Yv])
    del Xv, Yv, preds_nn, df_val
    gc.collect()

print (X.shape)
print (np.mean(np.abs(fold_scores)))

#nn = create_model_nn()
#nn.fit(X[:,2:], Y, epochs=4, batch_size=1024)
#nn.save('nn.h5')
#preds_nn = nn.predict(X[1000000:,2:])[:,0]
nn = create_model_nn2(X[:,0])
nn.fit([X[:,0], X[:,2:]], Y, epochs=4, batch_size=1024)
nn.save('nn.h5')
preds_nn = nn.predict([X[1000000:,0], X[1000000:,2:]])[:,0]
del nn
df_val = pd.DataFrame()
df_val['preds'] = preds_nn
df_val['time_id'] = X[1000000:,1]
df_val['Y'] = Y[1000000:]
r = df_val.groupby('time_id').apply(lambda x: pearsonr(df_val['Y'], df_val['preds'])[0]).mean()
print (r)

#nn2 = create_model_nn2(X[:,0])
#nn2.fit([X[:,0], X[:,1:]], Y, epochs=4, batch_size=1024)
#nn2.save('nn.h5')
#print (pearsonr(nn2.predict([X[1000000:,0], X[1000000:,1:]])[:,0], Y[1000000:])[0])

del X, Y, preds_nn, df_val#, nn2
gc.collect()

3-fold time split CV (val score is mean of fold_scores only, not full train)

4096, 512, 32 (with all 0.2s) relu: (-0.1320, 0.1339, 0.1345, 0.2165) val, LB 0.139

2048 relu, 1024 gelu, 512 gelu, 128 gelu, 32 relu (no dropouts):

original architecture with id_lookup layer: (-0.1336, 0.1406, 0.1456, 0.2296), LB 0.137

5X 512, 32 (with all 0.2s) relu:

10X 256, 32 (with all 0.2s) relu: (0.1190, -0.1496, 0.1491, -0.1630) val, LB 0.140

10X 256, 32 (with all 0.4s) relu: 